### Student Information
Name:

Student ID: 108048110

GitHub ID: MeganKuo0704

Kaggle name:

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2022-Lab2-master Repo](https://github.com/keziatamus/DM2022-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 15th 11:59 pm, Tuesday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 18th 11:59 pm, Friday)__. 

In [5]:
### Begin Assignment Here
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import os
import re
from itertools import chain
import tensorflow as tf
from keras.callbacks import CSVLogger
from sklearn.metrics import accuracy_score

In [ ]:
id = pd.read_csv('C:\\Users\\megan\\Desktop\\DSPMT\\DS\\Data Mining\\Lab2\\dataset\\data_identification.csv')
emo = pd.read_csv('C:\\Users\\megan\\Desktop\\DSPMT\DS\\Data Mining\Lab2\dataset\\emotion.csv')
tweets = pd.read_json('C:\\Users\\megan\\Desktop\\DSPMT\\DS\\Data Mining\\Lab2\\dataset\\tweets_DM.json', lines=True)

train_id = id[id.identification=='train']
train_emo = emo[emo.tweet_id.isin(train_id.tweet_id)]
tweets['_index'] = [tweets._source[i]['tweet']['tweet_id'] for i in range(0, tweets.shape[0])]
tweets['hashtags'] = [tweets._source[i]['tweet']['hashtags'] for i in range(0, tweets.shape[0])]
tweets['text'] = [tweets._source[i]['tweet']['text'] for i in range(0, tweets.shape[0])]

tweets['year'] = tweets._crawldate.apply(lambda x: x.split()[0].split('-')[0])
tweets['month'] = tweets._crawldate.apply(lambda x: x.split()[0].split('-')[1])
tweets['day'] = tweets._crawldate.apply(lambda x: x.split()[0].split('-')[2])
tweets['hour'] = tweets._crawldate.apply(lambda x: x.split()[1].split(':')[0])


In [9]:
tweets._crawldate = pd.to_datetime(tweets.iloc[:, 7:])
tweets._crawldate -= min(tweets._crawldate)

tweets.drop(['_source', '_type', 'year', 'month', 'day', 'hour'], axis=1, inplace=True)

In [11]:
tweets.to_pickle('tweets_clean.pkl')

In [12]:
tweets = pd.read_pickle('tweets_clean.pkl')
emo = pd.read_csv('C:\\Users\\megan\\Desktop\\DSPMT\DS\\Data Mining\Lab2\dataset\\emotion.csv')
id = pd.read_csv('C:\\Users\\megan\\Desktop\\DSPMT\\DS\\Data Mining\\Lab2\\dataset\\data_identification.csv')

In [25]:
train_id = id[id.identification=='train']
train_emo = emo[emo.tweet_id.isin(train_id.tweet_id)]

test_id = id[id.identification=='test']
test_df = tweets[tweets['_index'].isin(test_id.tweet_id)]


In [13]:
train_emo.columns = ['_index', 'emo']
train_df = pd.merge(tweets, train_emo, how='inner', on='_index')

test_df = train_df.emo

In [28]:
train_df

,_score,_index,_crawldate,hashtags,text,hour,emo
0,391,0x376b20,142 days 10:00:00,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",11,anticipation
1,433,0x2d5350,392 days 03:00:00,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",04,sadness
2,376,0x1cd5b0,388 days 22:00:00,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,23,fear
3,120,0x1d755c,161 days 03:00:00,"[authentic, LaughOutLoud]",@RISKshow @TheKevinAllison Thx for the BEST TI...,04,joy
4,1021,0x2c91a8,229 days 01:00:00,[],Still waiting on those supplies Liscus. <LH>,02,anticipation
...,...,...,...,...,...,...,...
1455558,94,0x321566,725 days 01:00:00,"[NoWonder, Happy]",I'm SO HAPPY!!! #NoWonder the name of this sho...,02,joy
1455559,627,0x38959e,90 days 07:00:00,[],In every circumtance I'd like to be thankful t...,08,joy
1455560,274,0x2cbca6,686 days 22:00:00,[blessyou],there's currently two girls walking around the...,23,joy
1455561,840,0x24faed,610 days 13:00:00,[],"Ah, corporate life, where you can date <LH> us...",14,joy


In [36]:
TAG_RE = re.compile(r'<[^>]+>')

def preprocess_text(sen):
    # Removing html tags
    sentence = TAG_RE.sub('', sen)
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

def tokenizer(data):
    tokenize_model = CountVectorizer(max_features = 500, tokenizer=nltk.word_tokenize)
    tokenize_model.fit(data['text'])
    token_data = tokenize_model.transform(data['text'])

    return token_data

In [39]:
preprocess_text(train_df.text[0])

'People who post add me on Snapchat must be dehydrated Cuz man that '

In [42]:
train_tokens = train_df.text.apply(lambda x: preprocess_text(x))
test_tokens = test_df.text.apply(lambda x: preprocess_text(x))

In [43]:
train_df.text = train_tokens
test_df.text = test_tokens

c:\Users\megan\python\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [48]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [62]:
scaler.fit(train_df._crawldate.to_numpy().reshape(train_df.shape[0], 1))
day_range = scaler.transform(train_df._crawldate.to_numpy().reshape(train_df.shape[0], 1))
train_df._crawldate = day_range

scaler.fit(test_df._crawldate.to_numpy().reshape(test_df.shape[0], 1))
day_range = scaler.transform(test_df._crawldate.to_numpy().reshape(test_df.shape[0], 1))
test_df._crawldate = day_range

In [49]:
scaler.fit(train_df._score.to_numpy().reshape(train_df.shape[0], 1))
score_range = scaler.transform(train_df._score.to_numpy().reshape(train_df.shape[0], 1))
train_df._score = score_range

scaler.fit(test_df._score.to_numpy().reshape(test_df.shape[0], 1))
score_range = scaler.transform(test_df._score.to_numpy().reshape(test_df.shape[0], 1))
test_df._score = score_range

In [50]:
train_df.to_pickle('train_df.pkl')
test_df.to_pickle('test_df.pkl')

In [6]:
train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_df = train_df.sample(frac=0.5)

In [9]:
train_df.shape

(727782, 7)

In [10]:
X_train, y_train = train_test_split(train_df, test_size = 0.3, random_state=42)

In [11]:
X_test = X_train.emo
y_test = y_train.emo

In [12]:
X_train.drop(['emo', '_index', 'hashtags'], inplace=True, axis=1)
y_train.drop(['emo', '_index', 'hashtags'], inplace=True, axis=1)

c:\Users\megan\python\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
import nltk

# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_500.fit(train_df['text'])

train_features = BOW_500.transform(X_train.text)
val_features = BOW_500.transform(y_train.text)

c:\Users\megan\python\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [14]:
val_features = val_features.astype('int32')
train_features = train_features.astype('int32')

In [15]:
X_train._crawldate = X_train._crawldate.astype('float32')
X_train._score = X_train._score.astype('float32')
y_train._crawldate = y_train._crawldate.astype('float32')
y_train._score = y_train._score.astype('float32')

c:\Users\megan\python\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [16]:
tmp = np.append(X_train._crawldate.to_numpy().reshape(X_train.shape[0], 1), np.array(X_train._score).reshape(X_train.shape[0], 1), axis=1)
train_features = np.append(train_features.toarray(), tmp, axis=1).astype('float32')
tmp = np.append(y_train._crawldate.to_numpy().reshape(y_train.shape[0], 1), np.array(y_train._score).reshape(y_train.shape[0], 1), axis=1)
val_features = np.append(val_features.toarray(), tmp, axis=1).astype('float32')

In [17]:
train_features.shape, X_test.shape, val_features.shape, y_test.shape

((509447, 502), (509447,), (218335, 502), (218335,))

In [18]:
target_features = BOW_500.transform(test_df.text)

In [19]:
tmp = np.append(np.array(test_df._crawldate).reshape(test_df.shape[0], 1), np.array(test_df._score).reshape(test_df.shape[0], 1), axis=1)
target_features = np.append(target_features.toarray(), tmp, axis=1)

In [20]:
label_encoder = LabelEncoder()
label_encoder.fit(train_df.emo)  # learn categorical labels

print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('test_df[0:7]:\n', train_df.emo[0:7])
print('\ntest_df.shape: ', train_df.emo.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

x_test = label_encode(label_encoder, X_test)
val_test = label_encode(label_encoder, y_test)


print('\n\n## After convert')
print('test_df[0:7]:\n', train_df.emo[0:7])
print('\ntest_df.shape: ', train_df.emo.shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
test_df[0:7]:
 1061508            fear
26052               joy
133593              joy
309260              joy
232670              joy
952244            trust
814660     anticipation
Name: emo, dtype: object

test_df.shape:  (727782,)


## After convert
test_df[0:7]:
 1061508            fear
26052               joy
133593              joy
309260              joy
232670              joy
952244            trust
814660     anticipation
Name: emo, dtype: object

test_df.shape:  (727782,)


In [21]:
# I/O check
input_shape = train_features.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  502
output_shape:  8


In [22]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Bidirectional
from keras.layers import Embedding, GRU, SimpleRNN, LSTM
from keras.layers import ReLU, Softmax
from keras.constraints import maxnorm
import keras.backend as K

In [43]:
lstm_train_features = train_features.reshape(, train_features.shape[0], train_features.shape[1])

In [79]:
train_features.shape

(101889, 1202)

In [45]:
lstm_val_features = val_features.reshape(1, val_features.shape[0], val_features.shape[1])

In [29]:
from keras_self_attention import SeqSelfAttention


model = keras.models.Sequential()
model.add(keras.layers.Embedding(input_dim=input_shape,
                                 output_dim=500,
                                 mask_zero=True))

model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=128,
                                                       return_sequences=True)))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(keras.layers.Dense(units=8))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'],
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 500)         601000    
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 256)        644096    
 nal)                                                            
                                                                 
 seq_self_attention_1 (SeqSe  (None, None, 256)        16449     
 lfAttention)                                                    
                                                                 
 dense_1 (Dense)             (None, None, 8)           2056      
                                                                 
Total params: 1,263,601
Trainable params: 1,263,601
Non-trainable params: 0
_________________________________________________________________


In [24]:
# input layer
# input layer
model = keras.Sequential()

model.add(Embedding(input_dim=input_shape, output_dim = 64, mask_zero=True))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32, return_sequences=False)))
model.add(Dense(8))

model.summary()

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          32128     
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 128)        66048     
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 8)                 520       
                                                                 
Total params: 139,912
Trainable params: 139,912
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
__________________________

In [71]:
from keras.layers import Attention, Embedding

In [45]:
epochs = 40 # total num of iterations
batch_size = 64 # pass 32 samples to the network afrom keras.models import Model

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=512)(X)  # 64
H1 = ReLU()(X_W1)
D1 = Dropout(0.3)(H1)

# 2nd hidden layer
H1_W2 = Dense(units=512)(D1)  # 64
H2 = ReLU()(H1_W2)
D2 = Dropout(0.3)(H2)

# 3nd hidden layer
H1_W3 = Dense(units=256)(D2)  # 64
H3 = ReLU()(H1_W3)
D3 = Dropout(0.3)(H3)

# 4nd hidden layer
H1_W4 = Dense(units=256)(D3)  # 64
H4 = ReLU()(H1_W4)
D4 = Dropout(0.3)(H4)

# 5nd hidden layer
H1_W5 = Dense(units=128)(D4)  # 64
H5 = ReLU()(H1_W5)

# output layer
H2_W1 = Dense(units=output_shape)(H5)  # 4
H11 = Softmax()(H2_W1)

model_output = H11

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
print(model.summary())

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1202)]            0         
                                                                 
 dense_23 (Dense)            (None, 512)               615936    
                                                                 
 re_lu (ReLU)                (None, 512)               0         
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_24 (Dense)            (None, 512)               262656    
                                                                 
 re_lu_1 (ReLU)              (None, 512)               0         
                                                                 
 dropout_13 (Dropout)        (None, 512)               0   

In [50]:
epochs = 10 # total num of iterations
batch_size = 64 # pass 32 samples to the network afrom keras.models import Model

model_input = Input(shape=(None, ))  # 500
X = model_input

# input layer
E1 = Embedding(input_dim=input_shape, output_dim = 64, mask_zero=True)(X)
att = SeqSelfAttention(attention_activation='sigmoid')(E1)
RNN_layer = SimpleRNN(64, return_sequences=False, activation='tanh')(att)
# output layer
O1 = Dense(units=output_shape, activation='softmax')(RNN_layer)  # 4

model_output = O1

model = Model(inputs=[model_input], outputs=[model_output])
# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
print(model.summary())

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_6 (Embedding)     (None, None, 64)          76928     
                                                                 
 seq_self_attention_6 (SeqSe  (None, None, 64)         4161      
 lfAttention)                                                    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 128)               24704     
                                                                 
 dense_29 (Dense)            (None, 64)                8256      
                                                                 
 dense_30 (Dense)            (None, 8)                 520       
                                                           

In [16]:
train_features = train_features.toarray()
val_features = val_features.toarray()

In [29]:
train_features = train_features.astype('float32')
val_features = val_features.astype('float32')

In [32]:
train_features.shape

(1018894, 500)

In [49]:
x_test.shape

(1018894, 8)

In [50]:
lstm_x_test = x_test.reshape(1, x_test.shape[0], x_test.shape[1])
lstm_val_test = val_test.reshape(1, val_test.shape[0], val_test.shape[1])

In [25]:
# csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 5 # total num of iterations
batch_size = 30 # pass 32 samples to the network at a time

# training!
history = model.fit(train_features, x_test, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    validation_data = (val_features, val_test))
print('training finish')

Epoch 1/5
16982/16982 [==============================] - 13816s 813ms/step - loss: 7.2408 - accuracy: 0.3539 - val_loss: 7.2419 - val_accuracy: 0.3558
Epoch 2/5
 8336/16982 [=============>................] - ETA: 5:43:32 - loss: 7.2479 - accuracy: 0.3543

In [35]:
val_test.shape

(218335, 8)

In [47]:
lstm_target_features = target_features.reshape(1, target_features.shape[0], target_features.shape[1])

In [47]:
## predict
target_pred = model.predict(target_features, batch_size=64)


In [48]:
target_pred = label_decode(label_encoder, target_pred)
target_pred[:5]

array(['anticipation', 'anticipation', 'joy', 'joy', 'joy'], dtype=object)

In [49]:
submission = pd.DataFrame(np.append(test_df['_index'].to_numpy().reshape(test_df.shape[0], 1), target_pred.reshape(test_df.shape[0], 1), axis=1), columns=['id', 'emotion'])
submission.to_csv('submission.csv', index=False)